# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [2]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [7]:
# Get the testCase
# imagePath, maskPath = radiomics.getTestCase('brain1')
# Patient01657_Plane3_1_of_2.png
imagePath = '/mnt/storage/fangyijie/SP_transthalamic/transthalamic/Patient01657_Plane3_1_of_2.png'
maskPath = '/mnt/storage/fangyijie/SP_transthalamic/annotations/Patient01657_Plane3_1_of_2_Annotation.png'
image_name = 'Patient01657_Plane3_1_of_2.png'

if imagePath is None or maskPath is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = os.path.join('exampleSettings', 'Params.yaml')
print('Image Path:', imagePath)
print('Mask Path:', maskPath)
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Image Path: /mnt/storage/fangyijie/SP_transthalamic/transthalamic/Patient01657_Plane3_1_of_2.png
Mask Path: /mnt/storage/fangyijie/SP_transthalamic/annotations/Patient01657_Plane3_1_of_2_Annotation.png
Parameter file, absolute path: /home/fangyijie/radiomics/exampleSettings/Params.yaml


### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [3]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(force2D=True)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


##### Method 2, hard-coded settings:

In [16]:
# First define the settings
settings = {}
settings['binWidth'] = 20
settings['sigma'] = [1, 2, 3]

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)  # ** 'unpacks' the dictionary in the function call

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1, 2, 3]}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [17]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeatureExtractor(binWidth=20, sigma=[1, 2, 3])  # Equivalent of code above

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)  # Still the default parameters
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default parameters

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 20, 'sigma': [1, 2, 3]}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [18]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableImageTypeByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.enabledImagetypes)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'firstorder': [], 'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares']}


##### Method 3, using a parameter file

In [8]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath, force2D=True, label=255)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'shape2D': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None}


### Extract features

image = sitk.ReadImage(ImageFilePath)Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [9]:
import SimpleITK as sitk
image = sitk.ReadImage(imagePath)
image_array = sitk.GetArrayFromImage(image)

mask = sitk.ReadImage(maskPath)
mask_array = sitk.GetArrayFromImage(mask)

In [10]:
import numpy as np
from PIL import Image

In [11]:
np.unique(image_array)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [12]:
np.unique(mask_array)

array([  0, 255], dtype=uint8)

In [13]:
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase

In [45]:
# image = sitk.ReadImage(imagePath, sitk.sitkInt8)
image = sitk.ReadImage(imagePath)
mask = sitk.ReadImage(maskPath)

In [31]:
image.GetDimension()

2

In [32]:
mask.GetDimension()

2

In [15]:
result = extractor.execute(imagePath, maskPath)

/home/fangyijie/miniconda3/envs/rad_user/lib/python3.11/site-packages/radiomics/imageoperations.py:125: RuntimeWarning: overflow encountered in scalar add
  highBound = maximum + 2 * binWidth


In [18]:
print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
print('')
print('Calculated features')
ind = 0
for key, value in six.iteritems(result):
    print('\t', key, ':', value)
    ind += 1

print(f'there are {ind} features extracted')

Result type: <class 'collections.OrderedDict'>

Calculated features
	 diagnostics_Versions_PyRadiomics : v3.0.1
	 diagnostics_Versions_Numpy : 2.0.0
	 diagnostics_Versions_SimpleITK : 2.3.1
	 diagnostics_Versions_PyWavelet : 1.6.0
	 diagnostics_Versions_Python : 3.11.9
	 diagnostics_Configuration_Settings : {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
	 diagnostics_Configuration_EnabledImageTypes : {'Original': {}}
	 diagnostics_Image-original_Hash : 0f66bd355b2bf7faeda36aefecbe78a59f20e836
	 diagnostics_Image-original_Dimensionality : 2D
	 diagnostics_Image-original_Spacing : (1.0, 1.0)
	 diagnostics_Image-original_Size : (959, 661)
	 diagnostics_Image-original_

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame([result], columns=result.keys())

In [21]:
df['image_name'] = image_name

In [22]:
df

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,image_name
0,v3.0.1,2.0.0,2.3.1,1.6.0,3.11.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0f66bd355b2bf7faeda36aefecbe78a59f20e836,2D,"(1.0, 1.0)",...,0.005278653730167794,3.984079551920479,8.96599921810588,35.740198260479126,2.2724494575125704,0.25398011201988024,0.11211492897365,0.4460256511160915,0.028637248438039623,Patient01657_Plane3_1_of_2.png


### Perform feature extraction on HC18

In [23]:
import pandas as pd

In [24]:
# Get the testCase
image_dir = '/mnt/storage/fangyijie/SP_transthalamic/transthalamic'
mask_dir = '/mnt/storage/fangyijie/SP_transthalamic/annotations'

paramPath = os.path.join('exampleSettings', 'Params.yaml')
print('Image Folder:', image_dir)
print('Mask Folder:', mask_dir)
print('Parameter file, absolute path:', os.path.abspath(paramPath))

Image Folder: /mnt/storage/fangyijie/SP_transthalamic/transthalamic
Mask Folder: /mnt/storage/fangyijie/SP_transthalamic/annotations
Parameter file, absolute path: /home/fangyijie/radiomics/exampleSettings/Params.yaml


In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
all_features = []
ind = 0

for img in os.listdir(image_dir):
    img_name = os.fsdecode(img)
    if img_name.endswith(".png"):
        mask_name = img_name.replace(".png", "_Annotation.png")
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, mask_name)
        # print(img_path)
        # print(mask_path)
        # print(img_name)
        
        try:
            feature = extractor.execute(img_path, mask_path)
            df = pd.DataFrame([feature], columns=feature.keys())
            df['image_name'] = img_name
            all_features.append(df)
            # print(df)
        except Exception as error:
            print("An exception occurred:", error)
            print(f"{img_path} is not processed")

        if ind % 50 == 0:
            print(f"{ind} images are processed")
        ind += 1

0 images are processed
50 images are processed
100 images are processed
150 images are processed
200 images are processed
An exception occurred: Calculation of GLCM Failed.
/mnt/storage/fangyijie/SP_transthalamic/transthalamic/Patient01481_Plane3_2_of_4.png is not processed
An exception occurred: Calculation of GLCM Failed.
/mnt/storage/fangyijie/SP_transthalamic/transthalamic/Patient01307_Plane3_1_of_2.png is not processed
250 images are processed
300 images are processed
350 images are processed
400 images are processed
450 images are processed
500 images are processed
550 images are processed
600 images are processed
650 images are processed
700 images are processed
An exception occurred: Calculation of GLCM Failed.
/mnt/storage/fangyijie/SP_transthalamic/transthalamic/Patient01347_Plane3_4_of_4.png is not processed
750 images are processed
800 images are processed
850 images are processed
900 images are processed
An exception occurred: Calculation of GLCM Failed.
/mnt/storage/fangy

In [27]:
all_features_df = pd.concat(all_features)

In [28]:
all_features_df.shape

(1552, 118)

### Save features locally

In [29]:
trg_dir = '/mnt/storage/fangyijie/SP_transthalamic/radiomics'
trg_csv = 'features.csv'

In [30]:
all_features_df.to_csv(os.path.join(trg_dir, trg_csv))